## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

edgeai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
import yaml
import shutil
from edgeai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#

print(f"INFO: current dir is: {os.getcwd()}")

INFO: current dir is: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


In [3]:
assert ('TIDL_TOOLS_PATH' in os.environ and 'LD_LIBRARY_PATH' in os.environ and
        'TARGET_SOC' in os.environ), "Check the environment variables"
print("TIDL_TOOLS_PATH=", os.environ['TIDL_TOOLS_PATH'])
print("LD_LIBRARY_PATH=", os.environ['LD_LIBRARY_PATH'])
print("TARGET_SOC=", os.environ['TARGET_SOC'])

TIDL_TOOLS_PATH= /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tools/AM68A/tidl_tools
LD_LIBRARY_PATH= /home/a0393608local/.pyenv/versions/3.10.14/envs/benchmark/lib/python3.10/site-packages/cv2/../../lib64:/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tools/AM68A/tidl_tools:
TARGET_SOC= AM68A


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [4]:
modelartifacts_tempdir_name = os.path.abspath('./work_dirs_custom')
modelartifacts_custom = os.path.join(modelartifacts_tempdir_name, 'modelartifacts')
print(f'INFO: clearing modelartifacts: {modelartifacts_custom}')
if os.path.exists(modelartifacts_custom):
    shutil.rmtree(modelartifacts_custom, ignore_errors=True)
#

INFO: clearing modelartifacts: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts


#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [5]:
settings = config_settings.CustomConfigSettings('./settings_import_on_pc.yaml',
                target_device = os.environ['TARGET_SOC'],
                modelartifacts_path=modelartifacts_custom,
                model_selection=None, model_shortlist=None,
                num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')

#### Download dataset if it doesn't exist

In [6]:
dataset_name='coco'
if not os.path.exists(f'{settings.datasets_path}/{dataset_name}'):
    print(f'INFO: downloading the dataset - {dataset_name}')
    interfaces.run_download_dataset(settings, dataset_name=dataset_name)
#

#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [7]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=True,
    num_frames=min(settings.calibration_frames,5000),
    name='coco'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=False, # can be set to True as well, if needed
    num_frames=min(settings.num_frames,5000),
    name='coco'
)

calib_dataset = datasets.COCODetection(**dataset_calib_cfg, download=True)
val_dataset = datasets.COCODetection(**dataset_val_cfg, download=True)


INFO:20250227-114805: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!

INFO:20250227-114807: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!


#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [8]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx models, tvmdlr for mxnet models.
session_name = constants.SESSION_NAME_TFLITERT
#session_name = constants.SESSION_NAME_ONNXRT
#session_name = constants.SESSION_NAME_TVMDLR

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

<class 'edgeai_benchmark.sessions.tflitert_session.TFLiteRTSession'>
{'advanced_options:quantization_scale_type': 4, 'tensor_bits': 8, 'accuracy_level': 1, 'debug_level': 0, 'inference_mode': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 12, 'advanced_options:calibration_iterations': 12, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': '', 'advanced_options:add_data_convert_ops': 3, 'advanced_options:c7x_firmware_version': '10_01_04_00', 'ti_internal_nc_flag': 83886080}


In [9]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [10]:
pipeline_configs = {
    'od-mlpefmnv1': dict(
        task_type='detection',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_tflite((300,300), (300,300), backend='cv2'),
        session=session_type(**tflite_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite'),
        postprocess=postproc_transforms.get_transform_detection_tflite(),
        metric=dict(label_offset_pred=datasets.coco_det_label_offset_90to90()),
        model_info=dict(metric_reference={'accuracy_ap[.5:.95]%':23.0})
    )
}
print(f"\nINFO: pipeline_configs={pipeline_configs}")


INFO: pipeline_configs={'od-mlpefmnv1': {'task_type': 'detection', 'calibration_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x7505d85d7790>, 'input_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x75063d298130>, 'preprocess': <edgeai_benchmark.preprocess.PreProcessTransforms object at 0x7505d85d69b0>, 'session': <edgeai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7505d85afd90>, 'postprocess': <edgeai_benchmark.postprocess.PostProcessTransforms object at 0x7505d85afa60>, 'metric': {'label_offset_pred': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 5

#### Model Compilation

This will take a few minutes. Please be patient...

In [11]:
# run the model compliation/import and inference
interfaces.run_benchmark_config(settings, work_dir, pipeline_configs)
run_dir = list(pipeline_configs.values())[0]['session'].get_run_dir()
print(f"\nINFO: compiled artifacts is in: {run_dir}")

# print the result:
with open(os.path.join(run_dir, "result.yaml")) as fp:
    result_yaml = yaml.safe_load(fp)
    result_dict = result_yaml['result']
    print(f"INFO: result - {result_dict}")
#

INFO: number of configs: 1


TASKS TOTAL=1, NUM_RUNNING=0:   0%|                                                                                                                                                                                                                                | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': [], 'COMPLETED': []}]


INFO:20250227-114809: starting - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite
INFO:20250227-114809: model_path - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite
INFO:20250227-114809: model_file - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite/model/ssd_mobilenet_v1_coco_20180128.tflite
INFO:20250227-114809: quant_file - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite/model/ssd_mobilenet_v1_coco_20180128_qparams.prototxt
Download done for /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite
Download done for /data/ssd/files/a0393

TASKS TOTAL=1, NUM_RUNNING=1:   0%|                                                                                                                                                                                                           | 0/1 [00:00<?, ?it/s, postfix={'RUNNING': ['od-mlpefmnv1:import'], 'COMPLETED': []}]

/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite/model/ssd_mobilenet_v1_coco_20180128.tflite

INFO:20250227-114809: running - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite

INFO:20250227-114809: pipeline_config - {'task_type': 'detection', 'calibration_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x7505d85d7790>, 'input_dataset': <edgeai_benchmark.datasets.coco_det.COCODetection object at 0x75063d298130>, 'preprocess': <edgeai_benchmark.preprocess.PreProcessTransforms object at 0x7505d85d69b0>, 'session': <edgeai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7505d85afd90>, 'postprocess': <edgeai_benchmark.postprocess.PostProcessTransforms object at 0x7505d85afa60>, 'metric': {'label_offset_pred': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14:

...


INFO:20250227-114844: infer completed  - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite - 8 sec
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


TASKS TOTAL=1, NUM_RUNNING=1:   0%|                                                                                                                                                                                                            | 0/1 [00:35<?, ?it/s, postfix={'RUNNING': ['od-mlpefmnv1:infer'], 'COMPLETED': []}]

DONE (t=0.11s).
Accumulating evaluation results...
DONE (t=0.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.025
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.023
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.026
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.014
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.066
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= lar

TASKS TOTAL=1, NUM_RUNNING=0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:36<00:00, 36.77s/it, postfix={'RUNNING': [], 'COMPLETED': ['od-mlpefmnv1']}]


INFO: compiled artifacts is in: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite
INFO: result - {'infer_path': 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite', 'accuracy_ap[.5:.95]%': 2.487425, 'accuracy_ap50%': 4.545896, 'num_subgraphs': 1}


#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [12]:
print()
out_dir = f'{work_dir}_package'
interfaces.package_artifacts(settings, work_dir, out_dir)


INFO: packaging artifacts to /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits_package please wait...
SUCCESS:20250227-114846: finished packaging - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite


#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [13]:
print(f'\nINFO: download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))


INFO: download the atricats files from the folder: /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/work_dirs_custom/modelartifacts/8bits_package
['artifacts.csv', 'artifacts.yaml', 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite.tar.gz', 'extract.sh']
